<a href="https://colab.research.google.com/github/LeoFernanndes/gambiarras/blob/main/GupyScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Teste de uma primeira requisição

In [4]:
req = requests.get("https://www.gupy.io/login/")

if req.status_code == 200:
    print('Requisição bem sucedida!')
    content = req.content

Requisição bem sucedida!


Antes de montar o loop, é importante entender a estrutura do html da página.
Vale começar dando uma olhada na documentação.

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [5]:
# dando uma investigada na estrutura pra montar o loop
soup = BeautifulSoup(content, 'html.parser')
soup.find_all(name='tr')[1].find_all(name='td')[1].a.get('href')

'https://4all.gupy.io/'

In [6]:
lista_empresas = []
lista_links = []

for tr in soup.find_all(name='tr'):  
  td_list = tr.find_all(name='td')
  
  if hasattr(td_list[1].a, 'href'): # escapar as linhas sem link    
    # empresa
    empresa = td_list[0].text
    lista_empresas.append(empresa)
    #link    
    link = td_list[1].a.get('href')
    lista_links.append(link)

df_empresas = pd.DataFrame({'Empresa': lista_empresas,
                            'Link': lista_links})

df_empresas

,Empresa,Link
0,4All,https://4all.gupy.io/
1,Banco ABC Brasil,https://abcbrasil.gupy.io/
2,AB InBev Chile,https://abinbevchile.gupy.io/
3,ACCT Consulting & Technology,https://acct.gupy.io/
4,Acesso Digital,https://acessodigital.gupy.io/
...,...,...
701,Zaffarinet,https://zaffarinet.gupy.io/
702,Zallpy,https://zallpy.gupy.io/
703,Zee-Dog,https://zee-dog.gupy.io/
704,Zenvia,https://zenvia.gupy.io/


In [7]:
lista_empresas = []
lista_vagas = []
lista_local = []
lista_remoto = []
lista_links = []

for i in range(df_empresas.shape[0]):

  nome_empresa = df_empresas.loc[i]['Empresa']
  link_empresa = df_empresas.loc[i]['Link']

  req_empresa = requests.get(link_empresa)
  if req_empresa.status_code != 200:
    continue

  content = req_empresa.content
  soup = BeautifulSoup(content, 'html.parser')

  for tr in soup.find_all(name='tr'):

    lista_empresas.append(nome_empresa)

    vaga = tr.find_all(name='td')[0].span.text.strip()
    lista_vagas.append(vaga)

    local = tr.find_all(name='td')[1].strong.text.replace('\n', '').strip()
    lista_local.append(local)

    remoto = 'remot' in tr.find_all(name='td')[1].strong.text.strip().lower()
    lista_remoto.append(remoto)

    link_relativo_vaga = tr.find_all(name='td')[0].a.get('href')
    link_vaga = link_empresa+link_relativo_vaga[1:]
    lista_links.append(link_vaga)

df_vagas = pd.DataFrame({
    'Empresa': lista_empresas,
    'Vaga': lista_vagas,
    'Local': lista_local,
    'Remoto': lista_remoto,
    'Link': lista_links
})

df_vagas

,Empresa,Vaga,Local,Remoto,Link
0,4All,Analista de Conteúdo [Phi],Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/492577?jobBoardSourc...
1,4All,Analista de Testes PL/SR [Food Rocket],Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/368291?jobBoardSourc...
2,4All,Designer [Phi],Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/492632?jobBoardSourc...
3,4All,Dev Back-end Node.JS Pleno [DX],Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/498229?jobBoardSourc...
4,4All,Dev Back-end Node.JS Pleno [Phi],Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/466720?jobBoardSourc...
...,...,...,...,...,...
14993,Zoom-Buscape,Product Owner,Rio de Janeiro and Remote,True,https://zoom-buscape.gupy.io/jobs/593438?jobBo...
14994,Zoom-Buscape,Software Engineer - Front End,Rio de Janeiro and Remote,True,https://zoom-buscape.gupy.io/jobs/471433?jobBo...
14995,Zoom-Buscape,Software Engineer - Full stack,Rio de Janeiro and Remote,True,https://zoom-buscape.gupy.io/jobs/471348?jobBo...
14996,Zoom-Buscape,Software Engineer - Mobile,Rio de Janeiro/RJ and Remote,True,https://zoom-buscape.gupy.io/jobs/192695?jobBo...


In [8]:
from datetime import date

dia = str(date.today())

df_vagas.to_excel(f'vagas_gupy_{dia}.xlsx', index=False, encoding='utf-8')

In [9]:
df_vagas[(df_vagas['Vaga'].str.contains('data|dados', case=False)) & (df_vagas['Remoto'] == True)] # exemplo de filtragem

,Empresa,Vaga,Local,Remoto,Link
191,Afya,Product Owner de Dados,São Paulo/SP and Remote,True,https://afya.gupy.io/jobs/644927?jobBoardSourc...
246,Agrotools,Data Science - Presencial e/ou Home Office,São José dos Campos/SP and Remote,True,https://agrotools.gupy.io/jobs/630415?jobBoard...
976,Amcom,Administrador de Banco de Dados - DBA,Blumenau/SC and Remote,True,https://amcom.gupy.io/jobs/639564?jobBoardSour...
1004,Amcom,Engenheiro de Dados,Blumenau/SC and Remote,True,https://amcom.gupy.io/jobs/638948?jobBoardSour...
1024,Amparo Saúde,Data Engineer Senior,São Paulo and Remote,True,https://amparosaude.gupy.io/jobs/504012?jobBoa...
...,...,...,...,...,...
14227,Will Bank,Remote Data Engineer Pleno,Remote Work,True,https://vemserwill.gupy.io/jobs/521806?jobBoar...
14228,Will Bank,Remote Data Engineer Sênior,São Paulo and Remote,True,https://vemserwill.gupy.io/jobs/573045?jobBoar...
14420,Via Varejo,Data Analyst CRM Sênior - Remoto,São Caetano do Sul/SP and Remote,True,https://viavarejo.gupy.io/jobs/634889?jobBoard...
14583,Vivo,Consultor (a) serviços implantação Big Data & IOT,São Paulo/SP and Remote,True,https://vivo.gupy.io/jobs/565587?jobBoardSourc...
